In [17]:
pip install pyautogen[gemini]

  Using cached google_api_core-2.19.1-py3-none-any.whl.metadata (2.7 kB)
  Using cached protobuf-5.27.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached proto_plus-1.24.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-4.25.4-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached pyasn1_modules-0.4.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_cloud_bigquery-3.25.0-py2.py3-none-any.whl.metadata (8.9 kB)
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
  Using cached googleapis_common_protos-1.63.2-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached google_cloud_core-2.4.1-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached grpc_google_iam_v1-0.13.1-py2.py3-none-any.whl.metadata (3.3 kB)
  Using cached google_crc32c-1.5.0-py3-none-any.whl
  Using cached pyasn1-0.6.0-py2.py3-none-any.whl.metadata (8.3 kB)
INFO: pip is looking at multiple versions of grpcio-status


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from IPython.display import Image, display

import autogen
from autogen.coding import LocalCommandLineCodeExecutor
import os

config_list = [
    {
        # Let's choose the Llama 3 model
        "model": "llama3-8b-8192",
        # Put your Groq API key here or put it into the GROQ_API_KEY environment variable.
        "api_key": os.environ.get("GROQ_API_KEY"),
        # We specify the API Type as 'groq' so it uses the Groq client class
        "api_type": "groq",
    }
]

In [ ]:
# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": 41,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        # the executor to run the generated code
        "executor": LocalCommandLineCodeExecutor(work_dir="coding"),
    },
)
# the assistant receives a message from the user_proxy, which contains the task description
chat_res = user_proxy.initiate_chat(
    assistant,
    message="""What date is today? Compare the year-to-date gain for META and TESLA.""",
    summary_method="reflection_with_llm",
)

In [ ]:
print("Chat history:", chat_res.chat_history)

print("Summary:", chat_res.summary)
print("Cost info:", chat_res.cost)

Chat history: [{'content': 'What date is today? Compare the year-to-date gain for META and TESLA.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "To get the current date, I'll use Python code to print the current date and time.\n\n```python\nimport datetime\nprint(datetime.date.today())\n```\n\nPlease execute this code to get the current date.\n\nOnce you have the current date, I can help you compare the year-to-date gain for META and TESLA. To do this, I'll need to collect some additional information. Can you please provide me with the current stock prices for META and TESLA? You can find this information on a financial website such as Yahoo Finance or Google Finance.\n\nOnce I have the current stock prices, I can calculate the year-to-date gain for both stocks and compare them.\n\nPlease provide me with the current stock prices for META and TESLA.", 'role': 'user', 'name': 'assistant'}, {'content': 'exitcode: 0 (execution succeeded)\nCode output: 2024-08-26\n', 'role': 'as

In [ ]:
# followup of the previous question
user_proxy.send(
    recipient=assistant,
    message="""Plot a chart of their stock price change YTD. Save the data to stock_price_ytd.csv, and save the plot to stock_price_ytd.png.""",
)

user_proxy (to assistant):

Plot a chart of their stock price change YTD. Save the data to stock_price_ytd.csv, and save the plot to stock_price_ytd.png.

--------------------------------------------------------------------------------
assistant (to user_proxy):

To plot a chart of their stock price change YTD, we'll need to get the historical stock prices for META and TESLA. We can use the `yfinance` library to get the historical prices.

Here's the code to get the historical prices and plot the chart:

```python
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# Get the historical prices for META and TESLA
meta_data = yf.download('META', start='2023-08-26', end='2024-08-26')
tesla_data = yf.download('TESLA', start='2023-08-26', end='2024-08-26')

# Calculate the YTD change
meta_ytd_change = (meta_data['Close'].iloc[-1] - meta_data['Close'].iloc[0]) / meta_data['Close'].iloc[0]
tesla_ytd_change = (tesla_data['Close'].iloc[-1] - tesla_data['Close'].iloc[0]) / 

In [ ]:
try:
    image = Image(filename="coding/stock_price_ytd.png")
    display(image)
except FileNotFoundError:
    print("Image not found. Please check the file name and modify if necessary.")

Image not found. Please check the file name and modify if necessary.


In [ ]:
# Path to your CSV file
file_path = "coding/stock_price_ytd.csv"
try:
    with open(file_path, mode="r", encoding="utf-8") as file:
        # Read each line in the file
        for line in file:
            # Split the line into a list using the comma as a separator
            row = line.strip().split(",")
            # Print the list representing the current row
            print(row)
except FileNotFoundError:
    print("File not found. Please check the file name and modify if necessary.")

File not found. Please check the file name and modify if necessary.
